In [135]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gp
import shapefile
import json

In [136]:
ny_subway = gp.read_file("../data/ny_subway_stations.geojson")

In [137]:
ny_subway.head()

,geometry,line,name,url
0,POINT (-73.99027199929638 40.68672833129772),F-G,Smith St & Bergen St At Ne Corner (To Manhatta...,http://www.mta.info/nyct/service/
1,POINT (-73.9906780008789 40.69372533216928),2-3-4-5-N-R,Court St & Montague St At Sw Corner,http://www.mta.info/nyct/service/
2,POINT (-73.99059199905471 40.69364233162101),2-3-4-5-N-R,Court St & Montague St At Sw Corner,http://www.mta.info/nyct/service/
3,POINT (-73.99253735666481 40.69439311951264),2-3-4-5-N-R,Clinton St & Montague St At Nw Corner,http://www.mta.info/nyct/service/
4,POINT (-73.96224891505983 40.66272735822238),B-Q-S,Flatbush Ave & Empire Blvd At Sw Corner,http://www.mta.info/nyct/service/


In [138]:
features = ny_subway.geometry.all()

In [139]:
#determine the unique subway lines
lines = set()
for s in ny_subway.line:
    lines.update(g for g in s.split('-'))
lines = sorted(lines)

In [140]:
#make a column for each line
for line in lines:
    ny_subway[line] = [line in station.split('-') for station in ny_subway.line]
         
ny_subway.head()

,geometry,line,name,url,1,2,3,4,5,6,...,F,G,J,L,M,N,Q,R,S,Z
0,POINT (-73.99027199929638 40.68672833129772),F-G,Smith St & Bergen St At Ne Corner (To Manhatta...,http://www.mta.info/nyct/service/,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
1,POINT (-73.9906780008789 40.69372533216928),2-3-4-5-N-R,Court St & Montague St At Sw Corner,http://www.mta.info/nyct/service/,False,True,True,True,True,False,...,False,False,False,False,False,True,False,True,False,False
2,POINT (-73.99059199905471 40.69364233162101),2-3-4-5-N-R,Court St & Montague St At Sw Corner,http://www.mta.info/nyct/service/,False,True,True,True,True,False,...,False,False,False,False,False,True,False,True,False,False
3,POINT (-73.99253735666481 40.69439311951264),2-3-4-5-N-R,Clinton St & Montague St At Nw Corner,http://www.mta.info/nyct/service/,False,True,True,True,True,False,...,False,False,False,False,False,True,False,True,False,False
4,POINT (-73.96224891505983 40.66272735822238),B-Q-S,Flatbush Ave & Empire Blvd At Sw Corner,http://www.mta.info/nyct/service/,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False


In [141]:
ny_subway.iloc[:, 4:].columns

subway_lines = ny_subway.iloc[:,4:]
subway_lines.drop("@", axis=1, inplace=True)
subway_lines.head()

# ny_subway.to_csv("processed_ny_subway_station.csv")

,1,2,3,4,5,6,6 Express,7,7 Express,A,...,F,G,J,L,M,N,Q,R,S,Z
0,False,False,False,False,False,False,False,False,False,False,...,True,True,False,False,False,False,False,False,False,False
1,False,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
2,False,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
3,False,True,True,True,True,False,False,False,False,False,...,False,False,False,False,False,True,False,True,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False


In [123]:
ny_subway["x"] = [i.x for i in ny_subway.geometry]
ny_subway["y"] = [i.y for i in ny_subway.geometry]

In [129]:
ny_subway.iloc[:, 1:].head()

,line,name,url,1,2,3,4,5,6,6 Express,...,J,L,M,N,Q,R,S,Z,x,y
0,F-G,Smith St & Bergen St At Ne Corner (To Manhatta...,http://www.mta.info/nyct/service/,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,-73.990272,40.686728
1,2-3-4-5-N-R,Court St & Montague St At Sw Corner,http://www.mta.info/nyct/service/,False,True,True,True,True,False,False,...,False,False,False,True,False,True,False,False,-73.990678,40.693725
2,2-3-4-5-N-R,Court St & Montague St At Sw Corner,http://www.mta.info/nyct/service/,False,True,True,True,True,False,False,...,False,False,False,True,False,True,False,False,-73.990592,40.693642
3,2-3-4-5-N-R,Clinton St & Montague St At Nw Corner,http://www.mta.info/nyct/service/,False,True,True,True,True,False,False,...,False,False,False,True,False,True,False,False,-73.992537,40.694393
4,B-Q-S,Flatbush Ave & Empire Blvd At Sw Corner,http://www.mta.info/nyct/service/,False,False,False,False,False,False,False,...,False,False,False,False,True,False,True,False,-73.962249,40.662727


In [132]:
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    # create a new python dict to contain our geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        # create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)
    
    return geojson

In [133]:
cols = ny_subway.columns.tolist()[1:]

In [134]:
geojson = df_to_geojson(ny_subway, cols, lat='y', lon='x')

# save the geojson result to a file
output_filename = 'dataset.geojson'
with open(output_filename, 'wb') as output_file:
    output_file.write('var dataset = {};'.format(json.dumps(geojson)))
    
# how many features did we save to the geojson file?
print('{} geotagged features saved to file'.format(len(geojson['features'])))

1904 geotagged features saved to file


In [76]:
names = ny_subway.name.values # get names of the station
subway_lines.iloc[:, :-2] = subway_lines.iloc[:, :-2].applymap(lambda x: int(x)) # turn boolean into intergets 
subway_lines['name'] = names

In [78]:
subway_lines.head()

,1,2,3,4,5,6,6 Express,7,7 Express,A,...,L,M,N,Q,R,S,Z,x,y,name
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,-73.990272,40.686728,Smith St & Bergen St At Ne Corner (To Manhatta...
1,0,1,1,1,1,0,0,0,0,0,...,0,0,1,0,1,0,0,-73.990678,40.693725,Court St & Montague St At Sw Corner
2,0,1,1,1,1,0,0,0,0,0,...,0,0,1,0,1,0,0,-73.990592,40.693642,Court St & Montague St At Sw Corner
3,0,1,1,1,1,0,0,0,0,0,...,0,0,1,0,1,0,0,-73.992537,40.694393,Clinton St & Montague St At Nw Corner
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,-73.962249,40.662727,Flatbush Ave & Empire Blvd At Sw Corner


In [79]:
subway_lines.to_csv("subway_lines.csv", encoding="utf-8", index=False)